# Dynamic scaLCA - combining dynamic material flow analysis with Life cycle assessment

## 1. Modules and Functions

### 1.1 Modules

In [1]:
import bw2data as bd
import bw2io as bi
from pathlib import Path
import bw2data as bd
import bw2calc as bc
import pandas as pd
import numpy as np
import scipy.stats
from os import chdir
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

### 1.2 Functions

In [2]:
def update_material_composition(secondary_steel_share):
    materials = {}
    steel_primary_share = 1 - secondary_steel_share
    materials['slt1'] = 0.546246 * steel_primary_share
    materials['slt2'] = 0.546246 * secondary_steel_share
    return materials

## 2. Life cycle assessment

### 2.1 Project set-up in Brightway

In [3]:
# Create project name
PROJECT_NAME = "Linking dMFA & LCA"

#Load Ecoinvent database
bi.restore_project_directory(
    fp='/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz',
    project_name=PROJECT_NAME,
    overwrite_existing=True
)
#Define project in brightway
bd.projects.set_current(PROJECT_NAME)

#Define database
db=bd.Database('ecoinvent-3.10-cutoff')

Restoring project backup archive - this could take a few minutes...
Restored project: Linking dMFA & LCA
08:41:01+0000 [info     ] Applying automatic update: 4.0 database search directories FTS5
08:41:01+0000 [info     ] Reindexing database ecoinvent-3.10-biosphere
08:41:01+0000 [info     ] Reindexing database ecoinvent-3.10-cutoff


### 2.2 Secondary steel production

In [4]:
# Define end-of-life treatment
steel_recycling_RER = bd.get_node(name='steel production, electric, chromium steel 18/8', location = 'RER')

""" what does this do?"""
my_edge = list(steel_recycling_RER.edges())[14]
my_edge.delete()

# Define scrap steel product nodes
scrap_stainless_steel = db.new_node(
    code='scrap stainless steel',
    name="scrap stainless steel",
    unit="kg",
    type=bd.labels.product_node_default,
)
#Save the node
scrap_stainless_steel.save()

# Define consumption edges
steel_recycling_RER.new_edge(
    amount=0.551, 
    type=bd.labels.consumption_edge_default,
    input=scrap_stainless_steel
).save()

### 2.3 Kettle production

In [43]:
summerschool_kettle_process['name']='summerschool kettle prod'

In [52]:
summerschool_kettle_process.save()

In [54]:
[n for n in db if 'electric kettle production' in n['name']]

['electric kettle production' (unit, GLO, None)]

In [58]:
summerschool_kettle_process['name']

'electric kettle production'

In [55]:
ecoinvent_kettle_process = bd.get_node(name='electric kettle production', location = 'GLO')
summerschool_kettle_process = ecoinvent_kettle_process.copy()

my_edge = list(summerschool_kettle_process.edges())[14]
#my_edge.delete()

In [6]:
db = bd.Database("ecoinvent-3.10-cutoff")  # 데이터베이스 객체 불러오기
matches = db.search("electric kettle production")  # 검색 실행

for node in matches:
    if node['location'] == 'GLO':
        print(node, node.get('reference product'))

for i, node in enumerate(matches):
    print(f"{i}: {node['name']}, location={node['location']}, reference product={node.get('reference product')}, id={node.id}")


'electric kettle production' (unit, GLO, None) electric kettle
'electric kettle production' (unit, GLO, None) electric kettle
0: electric kettle production, location=GLO, reference product=electric kettle, id=26455
1: electric kettle production, location=GLO, reference product=electric kettle, id=196545357926494208


In [34]:
matche

['electric kettle production' (unit, GLO, None),
 'electric kettle production' (unit, GLO, None)]

In [7]:
matches = db.search("electric kettle production")

# 첫 번째 노드를 선택
kettle = matches[0]
print(kettle, kettle.id, kettle.get("reference product"))


'electric kettle production' (unit, GLO, None) 26455 electric kettle


In [8]:
keywords = ['steel','chromium', 'nickel']
filtered_exchanges = []

for exc in summerschool_kettle_process.technosphere():
    if any(keyword in exc.input['name'] for keyword in keywords):
        filtered_exchanges.append(exc)

# 보기 좋게 출력
for exc in filtered_exchanges:
    print(f"{exc['amount']} {exc.input['unit']} '{exc.input['name']}' ({exc.input['location']})")

0.017734 kilogram 'market for chromium' (GLO)
-0.000558647372535729 kilogram 'market for nickel smelter slag' (CH)
-0.07037935262746428 kilogram 'market for nickel smelter slag' (RoW)
0.070938 kilogram 'market for nickel, class 1' (GLO)
-0.18281465556955923 kilogram 'market for scrap steel' (Europe without Switzerland)
-0.00023162033165852533 kilogram 'market for scrap steel' (CH)
-0.3825057240987823 kilogram 'market for scrap steel' (RoW)
0.547818 kilogram 'market for steel, low-alloyed, hot rolled' (GLO)


In [9]:
delete_targets = {
    'market for chromium',
    'market for nickel, class 1',
    'market for steel, low-alloyed, hot rolled', 
}

for exc in summerschool_kettle_process.edges():
    input_name = exc.input['name']
    
    if input_name in delete_targets and "waste" not in input_name.lower():
        print(f"Deleting: {input_name}, amount={exc['amount']}, location={exc.input['location']}")
        exc.delete()

Deleting: market for chromium, amount=0.017734, location=GLO
Deleting: market for nickel, class 1, amount=0.070938, location=GLO
Deleting: market for steel, low-alloyed, hot rolled, amount=0.547818, location=GLO


In [10]:
# 남아 있는 모든 엣지들의 input 이름 출력
remaining = [exc.input['name'] for exc in summerschool_kettle_process.edges()]
print("Remaining inputs:", remaining)

# 혹은 삭제 대상이 하나라도 남아 있는지 체크
for target in delete_targets:
    if target in remaining:
        print(f"⚠️ Still found: {target}")
    else:
        print(f"✅ Not found (deleted): {target}")


Remaining inputs: ['electric kettle production', 'market for brass', 'market for brass', 'market for compressed air, 800 kPa gauge', 'market for compressed air, 800 kPa gauge', 'market for copper, cathode', 'market group for electricity, low voltage', 'market for nickel smelter slag', 'market for nickel smelter slag', 'market for plastic processing factory', 'market for polyethylene, low density, granulate', 'market for polypropylene, granulate', 'market for polyvinylidenchloride, granulate', 'market for polyvinylidenchloride, granulate', 'market for scrap copper', 'market for scrap copper', 'market for scrap copper', 'market for scrap steel', 'market for scrap steel', 'market for scrap steel', 'market for silicone product', 'market for silicone product', 'market for textile, jute', 'market for waste polyethylene', 'market for waste polyethylene', 'market group for waste polyethylene', 'market for waste polyethylene', 'market for waste polyethylene', 'market for waste polyethylene', 'm

#### 2.3.1 Product and process definition

In [38]:
from bw2data import Database


# 2) kettle process
kettle = bd.get_node(
    name="market for electric kettle",
    location="GLO",
)
print(f"Product node: {kettle} (id={kettle.id})")

Product node: 'market for electric kettle' (unit, GLO, None) (id=8581)


In [56]:

# 3) kettle production 
kettle_production = bd.get_node(
    name="summerschool kettle prod",
    location="RER",
)
print(f"Process node: {kettle_production} (id={kettle_production.id})")


UnknownObject: 

#### 2.3.2 Technosphere flows kettle production

In [ ]:
""" Add all nodes which define the LCI"""

#electricity = db.new_node(
    #name='Electricity',
    #unit='kilowatt hour',
    #type=bd.labels.product_node_default,
#)
#electricity_production = db.new_node(
    #name='electricity production',
    #location='',
    #type=bd.labels.process_node_default,
#)

#electricity.save()
#electricity_production.save()

#cp_production = db.new_node(
    #code="cp-production",
    #name='copper production', #market for polypropylene, granulate
    #location='DE',
    #type=bd.labels.process_node_default,
#)
#cp = db.new_node(
    #code="cp",
    #name='copper',
    #unit="kilogram",
    #type=bd.labels.product_node_default,
#)

#cp_production.save()
#cp.save()

#cp_production.new_edge(
   # amount=1,
    #input=cp,
    #type=bd.labels.production_edge_default,
#).save()
#electricity_production.new_edge(
    #amount=1,
    #input=electricity,
    #type=bd.labels.production_edge_default,
#).save()   

materials = {
    slt1: 0.546246,
    slt2: 0,
    #cp: 0.017046,
    # ... other nodes and their corresponding values in kilograms
}

for material, amount in materials.items():
    kettle_production.new_edge(
        amount=amount,
        type=bd.labels.consumption_edge_default,
        input=material
    ).save()

#cp_production.new_edge(
    #amount=0.01389,  
    #uncertainty_type=5, 
    #minimum=0.012,  
    #maximum=0.016,  
    #type=bd.labels.consumption_edge_default,
    #input=electricity,
#).save()

#slt_production.new_edge(
    #amount=0.01389,  # 0.01389 kWh of electricity, in the papaer (MARCINKOWSKI, 2017)
    #uncertainty_type=5, 
    #minimum=0.012, 
    #maximum=0.016,
    #type=bd.labels.consumption_edge_default,
    #input=electricity,
#).save()

#### 2.3.3 Environmental flows kettle production

In [ ]:
""" is this a direct environmental emission during the kettle production?"""

#co2 = db.new_node(
#   name="Carbon Dioxide", 
#   context=('air',),
#   tags={'CAS Number': '124-38-9'},
#   unit='kilogram',
#    type=bd.labels.biosphere_node_default,
#)

#co2.save()

""" slt1_production not defined"""

# slt1_production.new_edge(
#     amount=4.62, 
#     uncertainty_type=5, 
#     minimum=4.158,
#     maximum=5.082, 
#     type=bd.labels.biosphere_edge_default,
#     input=co2,
# ).save()

# slt2_production.new_edge(
#     amount=0, 
#     uncertainty_type=5, 
#     minimum=0,
#     maximum=0, 
#     type=bd.labels.biosphere_edge_default,
#     input=co2,
# ).save()

## 3. Dynamic material flow analysis

### 3.1 Load and organise data

In [ ]:
# Load input data, inflow-driven model, Timeseries represents Irish purchases of 'hot water electronics' from 1980-2022
stock_flow_timeseries = pd.read_excel(r'WEEE_generated_Tool_IE.xlsm', sheet_name='POM')

# Extract the relevant rows and columns
stock_flow_timeseries = stock_flow_timeseries.iloc[20,6:-9]
stock_flow_timeseries = stock_flow_timeseries.to_frame()

#Define timesteps that are covered in the dataset
timesteps = range(1980,2023)

#Define timeseries dataset including the associated yeart
stock_flow_timeseries['year'] = timesteps
stock_flow_timeseries = stock_flow_timeseries.iloc[1:,:]
stock_flow_timeseries = stock_flow_timeseries.set_index('year')
stock_flow_timeseries = stock_flow_timeseries.rename(columns={20:'inflow'})

### 3.2 Define Lifetime curve

In [ ]:
#Define timesteps
time_max = stock_flow_timeseries.shape[0]
timesteps = np.arange(0, time_max)

# Weibull distributed survival curve representative for the Netherlands, Belgium and France (no data available for Ireland)
curve_shape = 1.73
curve_scale = 7.8
curve_surv = scipy.stats.weibull_min.sf(timesteps, curve_shape, 0, curve_scale)

#Visualise the result
plt.plot(curve_surv)

#Define data as float to align with later usage
curve_surv = curve_surv.astype(float)

### 3.3 Initiate normalized survival curve matrix

In [ ]:
# create survival curve matrix with placeholder zeros
curve_surv_matrix = pd.DataFrame(0.0, index=timesteps, columns=timesteps)

# populate the survival curve matrix with shifted curves, column by column using slices
for time in timesteps:
    curve_surv_matrix.loc[time:, time] = curve_surv[0:time_max - time]

### 3.4 Populate survival curve matrix with inflow values

In [ ]:
# create survival matrix with placeholder zeros
cohort_surv_matrix = pd.DataFrame(0.0, index=timesteps, columns=timesteps)

# multiply the inflow times the shifted curves to get the cohorts' behavior over time

for time in timesteps:
    cohort_surv_matrix.loc[:, time] = curve_surv_matrix.loc[:, time] * stock_flow_timeseries['inflow'].iloc[time]

# set row index to years instead of timesteps
cohort_surv_matrix.index = stock_flow_timeseries.index



### 3.5 Calculate stock, net addition to stock (NAS) and Outflow of products

In [ ]:
# calculate flows & stocks using the cohort_surv_matrix
stock_flow_timeseries['stock'] = cohort_surv_matrix.sum(axis=1)
stock_flow_timeseries['nas'] = np.diff(stock_flow_timeseries['stock'], prepend=0)  # prepending 0 assumes no initial stock
stock_flow_timeseries['outflow'] = stock_flow_timeseries['inflow'] - stock_flow_timeseries['nas']

#Visualize calculated flows and stocks
plt.plot(stock_flow_timeseries.index,stock_flow_timeseries['nas'], label = 'NAS')
plt.plot(stock_flow_timeseries.index,stock_flow_timeseries['inflow'], label = 'Inflow')
plt.plot(stock_flow_timeseries.index,stock_flow_timeseries['outflow'], label = 'outflow')
plt.legend()
plt.show()
plt.close()
plt.plot(stock_flow_timeseries.index,stock_flow_timeseries['stock'], label = 'stock')

### 3.6 Calculate material content in Inflow, NAS, Outflow and stock

In [ ]:
#Definde steel intensity as function of the inputs to the kettle process production
steel_intensity = 0.3653 #Because significant digits matter

#Calculate material content
steel_stock_flow_timeseries = stock_flow_timeseries*steel_intensity

# Visualize the comparison of the total inflow and the material contents in inflow
plt.plot(stock_flow_timeseries.index,stock_flow_timeseries['outflow'], label = 'outflow')
plt.plot(steel_stock_flow_timeseries.index,steel_stock_flow_timeseries['outflow'], label = 'steel outflow')
plt.legend()
plt.show()
plt.close()

### 3.7 Calculate secondary material availablility

In [ ]:
# Define static efficiency of steel recycling
steel_recycle_efficiency = 0.784444 #Because significant digits matter

# Calculate the secondary material availability of steel
secondary_steel_stock_flow_timeseries = steel_stock_flow_timeseries*steel_recycle_efficiency

#Visualise the steel outflow vs. the secondary steel outflow
plt.plot(steel_stock_flow_timeseries.index,steel_stock_flow_timeseries['outflow'], label = 'steel outflow')
plt.plot(secondary_steel_stock_flow_timeseries.index,secondary_steel_stock_flow_timeseries['outflow'], label = 'recycled steel')
plt.legend()
plt.show()
plt.close()

#Calculate the relative share of secondary material in the inflow for steel
secondary_steel_share = secondary_steel_stock_flow_timeseries['outflow']/steel_stock_flow_timeseries['inflow']


## 4. Integrate the dMFA results with the LCA

### 4.1 Update material composition in kettle year dependend kettle inventory

In [ ]:
def multiply_column_by_value(column, value):
    new_column = column * value
    return new_column.values

def calculate_remaining_fraction_column (column):
    new_column = 1 - column
    return new_column.values
    

In [ ]:
#Create df to store secondary steel values
secondary_steel_df = pd.DataFrame()
secondary_steel_df['years'] = stock_flow_timeseries.index

#Calculate secondary and primary steel share
secondary_steel_df['secondary steel share'] = secondary_steel_share
secondary_steel_df['primary steel share'] = calculate_remaining_fraction_column(secondary_steel_df['secondary steel share'])

#define original value steel
steel_primary_mass = 0.546246

#Call function to calculate new material composition and add to df
secondary_steel_df['updated secondary steel share'] = multiply_column_by_value(secondary_steel_df['secondary steel share'],steel_primary_share)


### 4.2 Calculate year dependend LCIAs of kettle lifecycle

In [ ]:
for value in secondary_steel_share_dict.items():
    materials = update_material_composition(value)
    primary_input = list(kettle_production.technosphere())[0]
    secondary_input = list(kettle_production.technosphere())[1]
    
    primary_input['amount'] = materials['slt1']
    primary_input.save()

    secondary_input['amount'] = materials['slt2']
    secondary_input.save()
    

    functional_unit, data_objs, _ = bd.prepare_lca_inputs({kettle: 1}, remapping=False)
    lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
    lca.lci()


In [ ]:
for value in np.array(secondary_steel_share):
    materials = update_material_composition(value)
    primary_input = list(kettle_production.technosphere())[0]
    secondary_input = list(kettle_production.technosphere())[1]
    
    primary_input['amount'] = materials['slt1']
    primary_input.save()

    secondary_input['amount'] = materials['slt2']
    secondary_input.save()
    

    functional_unit, data_objs, _ = bd.prepare_lca_inputs({kettle: 1}, remapping=False)
    lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
    lca.lci()


In [ ]:
    
    primary_input['amount'] = materials['slt1']
    primary_input.save()

    secondary_input['amount'] = materials['slt2']
    secondary_input.save()

    functional_unit, data_objs, _ = bd.prepare_lca_inputs({kettle: 1}, remapping=False)
    lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
    lca.lci()
    lca.inventory[lca.dicts.biosphere[co2.id], :].sum()
    lca.inventory[lca.dicts.biosphere[co2.id], :].sum()

In [ ]:
secondary_steel_share_dict

In [ ]:
my_edge['<some_key>'] = "<some_new_value>"
my_edge.save()


lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
lca.lci()
lca.inventory[lca.dicts.biosphere[co2.id], :].sum()

In [ ]:
primary_input = list(kettle_production.technosphere())[0]
primary_input

In [ ]:
secondary_input = list(kettle_production.technosphere())[1]
secondary_input

In [ ]:
primary_input['amount'] = 0.5
primary_input.save()
primary_input

In [ ]:
secondary_input['amount'] = 0.1
secondary_input.save()
secondary_input

In [ ]:
list(kettle_production.technosphere())

### Old environmental calculations

In [ ]:
functional_unit, data_objs, _ = bd.prepare_lca_inputs({kettle: 1}, remapping=False)